<a href="https://colab.research.google.com/github/kainat5008/Assignment-3/blob/main/ML_Assignment_Task5_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install flask pyngrok joblib numpy huggingface_hub

In [16]:
!pip install flask_ngrok

In [ ]:
!pip install flask pyngrok joblib numpy huggingface_hub
from flask import Flask, request, jsonify, render_template_string
import joblib
import numpy as np
from huggingface_hub import hf_hub_download
from pyngrok import ngrok
from google.colab import userdata
import os


ngrok.set_auth_token("2vffXqpQXoZ23eS7GhOEVqjXDez_NTwqaFkXepGEVgK6E7v9")  # Replace with your ngrok token

# HTML Template with basic styling
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Housing Price Predictor</title>
    <style>
        body { font-family: Arial; margin: 20px; }
        input, button { padding: 8px; margin: 5px 0; width: 300px; }
        button { background: #4CAF50; color: white; border: none; cursor: pointer; }
        .error { color: red; }
        .result { margin-top: 20px; font-weight: bold; }
    </style>
</head>
<body>
    <h1>California Housing Price Predictor</h1>
    <form action="/predict" method="post">
        <p>Enter 8 space-separated values (MedInc, HouseAge, etc.):</p>
        <input type="text" name="features" placeholder="0.1 20.0 5.0 1.0 500.0 6.0 40.0 -122.0" required>
        <button type="submit">Predict Price</button>
    </form>
    {% if error %}
        <p class="error">{{ error }}</p>
    {% endif %}
    {% if prediction %}
        <div class="result">Predicted Price: ${{ prediction }}</div>
    {% endif %}
</body>
</html>
"""

app = Flask(__name__)

# Load model and scaler with error handling
try:
    model_path = hf_hub_download(
        repo_id="keenu-5008/california-housing-regression",
        filename="batch_gd_model.pkl"
    )
    scaler_path = hf_hub_download(
        repo_id="keenu-5008/california-housing-regression",
        filename="scaler.pkl"
    )
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    print("Model and scaler loaded successfully!")
except Exception as e:
    print(f"Error loading model: {str(e)}")
    raise

@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get and validate input
        features = request.form['features'].strip().split()
        print(f"Received input: {features}")  # Debug

        if len(features) != 8:
            return render_template_string(
                HTML_TEMPLATE,
                error="Please enter exactly 8 numbers separated by spaces",
                prediction=None
            )

        # Convert to numpy array
        input_data = np.array([float(x) for x in features]).reshape(1, -1)
        print(f"Input array: {input_data}")  # Debug

        # Preprocess
        scaled_data = scaler.transform(input_data)
        scaled_data = np.c_[np.ones(scaled_data.shape[0]), scaled_data]  # Add bias term
        print(f"Preprocessed data: {scaled_data}")  # Debug

        # Predict
        prediction = model.predict(scaled_data)[0] * 1000  # Convert to dollars
        print(f"Raw prediction: {prediction}")  # Debug

        return render_template_string(
            HTML_TEMPLATE,
            prediction=f"{prediction:,.2f}",
            error=None
        )

    except ValueError as ve:
        return render_template_string(
            HTML_TEMPLATE,
            error=f"Invalid input: {str(ve)}",
            prediction=None
        )
    except Exception as e:
        print(f"Prediction error: {str(e)}")  # Debug
        return render_template_string(
            HTML_TEMPLATE,
            error=f"Server error: {str(e)}",
            prediction=None
        )

# Start the app
if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(f"\n⭐ Access your app at: {public_url}\n")
    app.run()

Model and scaler loaded successfully!

⭐ Access your app at: NgrokTunnel: "https://e1ab-35-233-250-200.ngrok-free.app" -> "http://localhost:5000"

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [13/Apr/2025 11:15:16] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Apr/2025 11:15:17] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Apr/2025 11:15:20] "POST /predict HTTP/1.1" 200 -


Received input: ['8']


INFO:werkzeug:127.0.0.1 - - [13/Apr/2025 11:15:47] "POST /predict HTTP/1.1" 200 -


Received input: ['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8']
Input array: [[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8]]
Preprocessed data: [[  1.          -1.9854439   -2.2542469   -2.15106165  -1.60822358
   -1.25411212  -0.21565696 -16.35455373  60.02329589]]
Prediction error: 'numpy.ndarray' object has no attribute 'predict'


INFO:werkzeug:127.0.0.1 - - [13/Apr/2025 11:16:15] "POST /predict HTTP/1.1" 200 -


Received input: ['0.1', '20.0', '5.0', '1.0', '500.0', '6.0', '40.0', '-122.0']
Input array: [[ 1.00e-01  2.00e+01  5.00e+00  1.00e+00  5.00e+02  6.00e+00  4.00e+01
  -1.22e+02]]
Preprocessed data: [[ 1.         -1.9854439  -0.6830824  -0.18231247 -0.22318659 -0.81480661
   0.25072898  2.03915078 -1.20548375]]
Prediction error: 'numpy.ndarray' object has no attribute 'predict'


INFO:werkzeug:127.0.0.1 - - [13/Apr/2025 11:16:28] "POST /predict HTTP/1.1" 200 -


Received input: ['0.1', '20.0', '5.0', '1.0', '500.0', '6.0', '40.0', '-122.0']
Input array: [[ 1.00e-01  2.00e+01  5.00e+00  1.00e+00  5.00e+02  6.00e+00  4.00e+01
  -1.22e+02]]
Preprocessed data: [[ 1.         -1.9854439  -0.6830824  -0.18231247 -0.22318659 -0.81480661
   0.25072898  2.03915078 -1.20548375]]
Prediction error: 'numpy.ndarray' object has no attribute 'predict'
